# Quantum teleportation using feed-forward

The goal of this notebook is to use perceval's feed-forward ability to demonstrate the Quantum teleportation algorithm on a photonic simulated circuit using dual rail encoding.

## Definition of the problem

The idea of the algorithm is the following:

Say that Alice has a generic qubit of the form

$$|\psi\rangle = \alpha |0\rangle + \beta |1> $$

that she wants to send to a distant receiver called Bob. Since Bob is distant, we want to avoid transporting physical systems from Alice to Bob.

Before the start of the algorithm, Alice and Bob need to share a maximally entangled Bell state. For this example, we will use the form

$$|\Psi\rangle = \frac{1}{\sqrt{2}} (|00\rangle + |11\rangle)$$

The first qubit is accessible to Alice and the second to Bob.

The total state is then

$$|\psi\rangle \otimes \Psi\rangle = (\alpha |0\rangle + \beta |1\rangle) \otimes \frac{1}{\sqrt{2}} (|00\rangle + |11\rangle)$$

The algorithm is the following:
Alice performs a CNOT using the first qubit as control and the second as target, then applies a H gate to the first qubit.

At the end, the total state can be written as

\begin{align}
    |final\rangle &= \frac{1}{2}|00\rangle \otimes (\alpha |0\rangle + \beta |1>) \\
                  &+ \frac{1}{2}|01\rangle \otimes (\beta |0\rangle + \alpha |1>) \\
                  &+ \frac{1}{2}|10\rangle \otimes (\alpha |0\rangle - \beta |1>) \\
                  &+ \frac{1}{2}|11\rangle \otimes (- \beta |0\rangle + \alpha |1>)
\end{align}

Then Alice needs to measure her two qubits and send the results to Bob using a classical channel.
First, if the second qubit is measured to be 1, Bob needs to apply a NOT gate to his qubit.
Then, if the first qubit is measured to be 1, Bob needs to apply a Z gate to his qubit.

After these operations, Bob's qubit is guaranteed to be the original qubit of Alice $|\psi\rangle$.

## Translation to Perceval

In [ ]:
import numpy as np

import perceval as pcvl
from perceval import catalog

### Starting state

First, we need to create the input state $|\psi\rangle \otimes |\Psi\rangle$ for this algorithm. We choose $\alpha$ and $\beta$ such that we can differentiate them using probabilities.

In [ ]:
# Creation of the qubit to transmit

alpha = .2  # Arbitrarily chosen with different modulus
beta = .1 + .3j
# alpha |0> + beta |1> in dual rail encoding
to_transmit = alpha * pcvl.BasicState([1, 0]) + beta * pcvl.BasicState([0, 1])
to_transmit.normalize()

alpha = to_transmit[pcvl.BasicState([1, 0])]  # Normalized
beta = to_transmit[pcvl.BasicState([0, 1])]

print(to_transmit)

In [ ]:
# Creation of the quantum channel
sg = pcvl.StateGenerator(pcvl.Encoding.DUAL_RAIL)
bell_state = sg.bell_state("phi+")
print(bell_state)

In [ ]:
input_state = to_transmit * bell_state

### Circuit

Now we need to define the circuit on which the operations will take place. Since we need to use gates and feed-forward components, we need to use a `Processor` object.

First, we define the photonic circuit that applies on the qubits.

In [ ]:
p = pcvl.Processor("SLOS", 6)
p.add(0, catalog["heralded cnot"].build_processor())
p.add(0, pcvl.BS.H());

Now we need to add the feed-forwarded components. For this purpose, Perceval uses two configurators that link measures to circuits or processors.

Both of them need to be defined by the number of modes they measure, the distance between the measured modes and the circuit they configure (this is an integer called `offset`), and a default configuration that is used whenever a measure does not befall into one of the defined cases.

The measured modes need to be classical modes. Thus, we need to add detectors before adding the configurators.

For the NOT gate, this gate corresponds to a permutation for a dual rail encoding if we measure $|1\rangle$, or an empty circuit if we measure $|0\rangle$. Thus, we are going to use a `FFCircuitProvider` as it links a measured state to a circuit or a processor.

In [ ]:
# 2 measured modes
# offset = 0 means that there is 0 empty modes between the measured modes and the circuit
# the default circuit is an empty circuit
ff_not = pcvl.FFCircuitProvider(2, 0, pcvl.Circuit(2))

# Now if we measure a logical state |1>, we need to perform a permutation of the modes
ff_not.add_configuration([0, 1], pcvl.PERM([1, 0]))

# Add perfect detectors to the modes that will be measured
p.add(2, pcvl.Detector.pnr())
p.add(3, pcvl.Detector.pnr())
p.add(2, ff_not);

The Z gate corresponds to a $\pi$ shift on the second mode. Thus, we are going to use a `FFConfigurator` that uses a parametrized circuit and links the measured states to a mapping of values for these parameters.

In [ ]:
phi = pcvl.P("phi")
# Like Circuits and Processors, we can chain the `add` methods
ff_z = pcvl.FFConfigurator(2, 3, pcvl.PS(phi), {"phi": 0}).add_configuration([0, 1], {"phi": np.pi})

p.add(0, pcvl.Detector.pnr())
p.add(1, pcvl.Detector.pnr())
p.add(0, ff_z);

We can check that we defined correctly our processor. Note that using the `recursive=True` flag, we can expose the inner circuit of the `FFConfigurator`.

In [ ]:
pcvl.pdisplay(p, recursive=True)

### Simulation

Now that we have both the input state and the processor, we can run the algorithm and check that it works.

In [ ]:
p.min_detected_photons_filter(3)

# Since we use a custom input state, we have to add the heralds from the heralded cnot
input_state *= pcvl.BasicState([1, 1])

p.with_input(input_state)

res = p.probs()
print(res)

Notice that when using feed-forward, the performance indicators are replaced by a single indicator "global_perf", which represents the probability that an output state checks all requirements. In our case, this corresponds to the CNOT performance: $2 / 27 \approx 0.074$.

For the results, we don't need to know what was measured by Alice, so we need to squash the resulting probabilities to keep only the two last modes.

In [ ]:
bsd = pcvl.BSDistribution()
for state, prob in res["results"].items():
    bsd[state[4:]] += prob

print(bsd)

We can now check that the resulting probabilities correspond to the probabilities of measure of the original qubit.

In [ ]:
prob_0 = abs(alpha) ** 2
prob_1 = abs(beta) ** 2

print("|0>:", prob_0, "|1>", prob_1)